In [ ]:
#!pip install transformers

In [ ]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

### 1. What is a pipeline in Hugging Face Transformers? What does it abstract away?
A **pipeline** is a high-level API that lets you run common ML tasks easily.  
It abstracts away:
- model downloading and loading  
- tokenization and preprocessing  
- postprocessing and decoding  
- device handling (CPU/GPU)

---

### 2. Three other available pipeline tasks
- summarization  
- translation  
- question-answering  

---

### 3. What happens if you don’t specify a model? How to specify one?
- If no model is specified, the pipeline automatically loads a **default model** for that task.  
- To specify a model:

```python
from transformers import pipeline
classifier = pipeline(
    "text-classification",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)


In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


### 1. What is the default model used for `text-classification`?
The default model is **distilbert-base-uncased-finetuned-sst-2-english**.

---

### 2. What dataset was this model fine-tuned on?  
It was fine-tuned on the **SST-2 (Stanford Sentiment Treebank)** dataset.  
This dataset contains short movie reviews, so the model works **best on short, simple, sentiment-focused text** (e.g., reviews, brief opinions).

---

### 3. What does the `score` field represent? What is its range?  
- The **score** is the model’s **probability/confidence** for the predicted label.  
- It ranges from **0 to 1**, where values closer to 1 mean higher confidence.

---

### 4. Challenge: Find a different text-classification model that classifies emotions  
One example is: **j-hartmann/emotion-english-distilroberta-base**  
(It predicts multiple emotions such as joy, anger, sadness, fear, etc.)


In [ ]:
import pandas as pd

outputs = classifier(text)
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.901546


In [ ]:
ner_tagger = pipeline("ner", aggregation_strategy="simple")
outputs = ner_tagger(text)
pd.DataFrame(outputs)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


,entity_group,score,word,start,end
0,ORG,0.879010,Amazon,5,11
1,MISC,0.990859,Optimus Prime,36,49
2,LOC,0.999755,Germany,90,97
3,MISC,0.556570,Mega,208,212
4,PER,0.590256,##tron,212,216
5,ORG,0.669692,Decept,253,259
6,MISC,0.498349,##icons,259,264
7,MISC,0.775362,Megatron,350,358
8,MISC,0.987854,Optimus Prime,367,380
9,PER,0.812096,Bumblebee,502,511


### 1. What does `aggregation_strategy="simple"` do in the NER pipeline?
`aggregation_strategy="simple"` merges tokens that belong to the **same entity** into a single, combined entity prediction.  
Without it, you would get predictions **token by token**, even if multiple tokens represent the same word or name.

---

### 2. What do the entity types mean? (ORG, MISC, LOC, PER)
These are the standard **CoNLL-2003 NER tags**:

- **ORG** — Organizations (companies, teams, institutions)  
- **MISC** — Miscellaneous entities (nationalities, events, other proper nouns)  
- **LOC** — Locations (countries, cities, regions)  
- **PER** — Persons (people’s names)

---

### 3. Why do some words appear with `##` prefixes (e.g., `##tron`, `##icons`)?
The `##` indicates that the token is a **subword piece**, not a full standalone word.  
This comes from **WordPiece tokenization**, where rare or unknown words are split into smaller chunks to handle vocabulary limits.

Example:  
`Megatron → Mega` + `##tron`

---

### 4. Why might the model split "Megatron" and "Decepticons" incorrectly? What does this tell us?
These words are **not common in the model’s training data**, so the tokenizer does not have complete tokens for them.  
Instead, it breaks them into subwords.

This indicates:
- The model was trained on **news text** (CoNLL-2003), not fictional names or Transformers-universe terms.  
- NER models perform worse on **domains they weren't trained on**.

---

### 5. Challenge: What is the CoNLL-2003 dataset?
The **CoNLL-2003** dataset is a widely used NER benchmark containing:
- news articles from **Reuters**  
- annotated with **PER, ORG, LOC, MISC** entity labels  

The model `dbmdz/bert-large-cased-finetuned-conll03-english` is trained specifically on this dataset.

---

### 6. 🤔 How might the choice of tokenizer affect NER performance?
- A tokenizer with **limited vocabulary** for your domain will split names into many subwords → poorer NER accuracy.  
- A domain-specific tokenizer (e.g., biomedical, legal, social media) leads to **better entity boundaries** and fewer incorrect splits.  
- Tokenizers trained on **cased vs. uncased** text also affect detection of proper nouns.

In short: **tokenizers shape how text is segmented, and NER depends heavily on correct segmentation**.


In [ ]:
reader = pipeline("question-answering")
question = "What does the customer want?"
outputs = reader(question=question, context=text)
pd.DataFrame([outputs])

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


,score,start,end,answer
0,0.631292,335,358,an exchange of Megatron


### 1. What type of question answering is this?
This is **extractive** question answering.  
Extractive QA selects a span of text **directly from the context**, rather than generating new words.

---

### 2. What do the start and end indices represent? Why are they important?
The model predicts:
- **start index** → where the answer begins in the context  
- **end index** → where the answer ends  

These indices allow the model to **slice out the exact answer span** from the input passage.  
Without them, the system wouldn't know which words form the answer.

---

### 3. What is the SQuAD dataset?
The model **distilbert-base-cased-distilled-squad** is fine-tuned on **SQuAD (Stanford Question Answering Dataset)**.  
SQuAD contains:
- Wikipedia articles  
- Human-written questions  
- Answers that are **continuous spans** of text in the passage  

It is a benchmark for **extractive** QA.

---

### 4. A question the model *cannot* answer based on the text — why?
Example:  
**“Why did the author write this passage?”**  
or  
**“What might happen next?”**

The model would fail because:
- extractive QA **cannot infer information not explicitly stated**  
- it only selects text spans from the provided context  
- it has **no reasoning beyond the content given**

---

### 5. Challenge: Extractive vs. Generative QA + an example model
**Extractive QA:**  
- Finds and returns a **substring** from the context  
- Example: `distilbert-base-cased-distilled-squad`

**Generative QA:**  
- **Generates** the answer using a language model  
- Not limited to the exact wording in the text  
- Can rewrite, summarize, or infer

**Example generative QA model:**  
- `google/flan-t5-base`  
- `meta-llama/Llam


### 1. Extractive vs. Abstractive Summarization
- **Extractive summarization** selects *existing sentences or phrases* from the text and stitches them together.
- **Abstractive summarization** *generates new sentences*, paraphrasing the content in its own words (like a human).

---

### 2. What is the default summarization model?
The default model is **facebook/bart-large-cnn**.

#### Is it extractive or abstractive?
It is **abstractive** — BART is a seq-to-seq generator that creates new text.

#### What architecture does it use?
It uses the **BART architecture** (denoising autoencoder with encoder–decoder structure), which is based on the Transformer.

#### What dataset was it trained on?
It is fine-tuned on the **CNN/DailyMail** news summarization dataset.

---

### 3. What do `max_length` and `min_length` control?
- **max_length** → the *maximum* length of the generated summary (in tokens)
- **min_length** → the *minimum* length of the generated summary

If **min_length > max_length**, the model will raise an error because the constraints cannot be satisfied.

---

### 4. What does `clean_up_tokenization_spaces=True` do?
It removes artifacts created during tokenization, such as:
- extra spaces  
- spacing around punctuation  
- leftover tokenization markers  

This is useful because abstractive summaries can contain small spacing errors due to subword tokenization.

---

### 5. Challenge — Find two summarization models

#### **A. Model optimized for short texts (news)**
**facebook/bart-large-cnn**
- **Architecture:** BART (encoder–decoder Transformer)
- **Good for:** Short news articles (a few paragraphs)
- **Training data:** CNN/DailyMail

#### **B. Model that can handle long documents**
**google/pegasus-large** or **google/pegasus-cnn_dailymail**
- **Architecture:** PEGASUS (Transformer encoder–decoder with gap-sentence pretraining)
- **Good for:** Much longer documents (research papers, multi-page text)
- **Training data:** Large mixture of news + web documents, depending on the variant (e.g., HugeNews dataset)

**Comparison:**
- **BART** is strong on short, concise summaries and is widely used for news.
- **PEGASUS** is designed to handle *longer sequences* and excels at multi-document or long-document summarization thanks to its pretraining objective.

